In [1]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests
import numpy
from scipy import stats
import math
from scipy.stats import ttest_ind
import tqdm

In [2]:
starting_directory = '/Users/Joseph/Desktop'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

FileChooser(path='/Users/Joseph/Desktop', filename='', title='', show_hidden=False, select_desc='Select', chan…

In [3]:
tap_chooser=FileChooser(starting_directory)
display(tap_chooser)

FileChooser(path='/Users/Joseph/Desktop', filename='', title='', show_hidden=False, select_desc='Select', chan…

In [4]:
Screen='Glia_Genes_Screen'
folder_path=baseline_chooser.selected_path
print(folder_path)

/Users/Joseph/Desktop/MWT_Datasets


In [5]:
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0).drop(columns=['index'])
print(baseline_output.head())
# X=baseline_output.drop(columns=['Time','n','Number','Tap','Puff','x','y','plate','dataset','Gene','Allele'])
# print(X)

      Time   n  Number  Instantaneous Speed  Interval Speed   Bias   Width  \
0  490.037  52      41               0.0319          0.0285  0.075  0.0906   
1  490.077  52      41               0.0361          0.0343  0.075  0.0922   
2  490.115  52      41               0.0323          0.0333  0.075  0.0935   
3  490.154  52      41               0.0335          0.0290  0.075  0.0915   
4  490.197  52      41               0.0337          0.0317  0.050  0.0909   

   Length      Area  Angular Speed  Aspect Ratio  Kink  Curve    Crab  \
0  0.8885  0.100122            1.9         0.262  40.5   26.4  0.0065   
1  0.8920  0.101651            2.1         0.264  40.1   26.4  0.0052   
2  0.8905  0.102931            2.2         0.262  39.9   26.2  0.0040   
3  0.8868  0.100922            2.5         0.259  40.4   26.3  0.0049   
4  0.8880  0.100193            2.4         0.254  41.1   26.7  0.0044   

  Plate_id      Date             Screen dataset Gene Allele  
0    B1210  20220919  Glia_Gen

In [6]:
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)
print(tap_output.head())

      time  dura   dist      prob     speed  plate      Date Plate_id  \
0  599.993  2.67  0.529  0.857143  0.198127      1  20220919    B1210   
1  609.924  2.05  0.479  0.852941  0.233659      1  20220919    B1210   
2  619.982  1.86  0.386  0.488372  0.207527      1  20220919    B1210   
3  629.946  1.87  0.371  0.500000  0.198396      1  20220919    B1210   
4  639.927  1.94  0.384  0.382979  0.197938      1  20220919    B1210   

              Screen  taps dataset Gene Allele  
0  Glia_Genes_Screen     1      N2   N2     N2  
1  Glia_Genes_Screen     2      N2   N2     N2  
2  Glia_Genes_Screen     3      N2   N2     N2  
3  Glia_Genes_Screen     4      N2   N2     N2  
4  Glia_Genes_Screen     5      N2   N2     N2  


In [ ]:
# test=tap_output.dropna()
# print(test[test.dataset=='mod-5_n822'].taps.unique())

In [7]:
PD_tap=tap_output
PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")

print(PD_first_tap)
print("-"*55)
print(PD_last_three_taps)
print("-"*55)
# print(PD_recov_taps)

print("-"*55)
PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
# print(PD_final_taps)
PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")


print("PD_final_taps")
print(PD_final_taps)
print("-"*55)


PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
# print(PD_final_taps)
# print(PD_first_tap)
PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
PD_habit_levels = PD_habit_levels.dropna()

print('PD_habit_levels')
print(PD_habit_levels)
print("-"*55)
# print(PD_habit_levels)
PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])

print('final version PD_habit_levels')
print(PD_habit_levels)
print("-"*55)

PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
print(PD_recov_taps)
print("-"*55)
PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')

# PD_habituation = PD_habit_levels
PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y'])
# PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y'])
PD_habituation = PD_habituation.dropna()
# print(PD_habituation)
# PD_habituation['recovery_dura']=0
# PD_habituation['recovery_prob']=0
# PD_habituation['recovery_speed']=0
# PD_habituation['dist']=0
# PD_habituation['recov_prob']=0
# PD_habituation['recov_dura']=0
# PD_habituation['recov_speed']=0
PD_habituation['recovery_dura']=PD_habituation.init_dura-PD_habituation.recov_dura
PD_habituation['recovery_prob']=PD_habituation.init_prob-PD_habituation.recov_prob
PD_habituation['recovery_speed']=PD_habituation.init_speed-PD_habituation.recov_speed
tap_data=PD_habituation.dropna() 
tap_data=PD_habituation
print("-"*55)
print(tap_data)
# print(PD_recov_taps)
# print(PD_first_tap)
# print(PD_last_three_taps)
print("-"*55)
# print(PD_habit_levels)

        time  dura   dist      prob     speed  plate      Date Plate_id  \
0    599.993  2.67  0.529  0.857143  0.198127      1  20220919    B1210   
1    599.980  2.32  0.525  0.944444  0.226293      2  20220811    C0811   
2    599.986  2.82  0.686  0.857143  0.243262      3  20220811    A0811   
3    599.991  2.73  0.698  0.958333  0.255678      4  20220811    C0811   
4    599.979  2.99  0.798  0.875000  0.266890      5  20220811    A0811   
..       ...   ...    ...       ...       ...    ...       ...      ...   
559  599.976  2.69  0.761  0.818182  0.282900      5  20221105    A1105   
560  599.991  2.63  0.730  0.916667  0.277567      6  20221105    B1105   
561  599.986  2.88  0.772  0.833333  0.268056      7  20221105    C1105   
562  599.961  2.97  0.701  0.842105  0.236027      8  20221105    C1105   
563  599.979  2.92  0.802  0.875000  0.274658      9  20221105    B1105   

                Screen  taps      dataset   Gene Allele  
0    Glia_Genes_Screen     1           N2

### The below cell reads in file from online repository (github) - example in case this is possible in the future

In [ ]:
# # tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
# tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# # s=requests.get(tap_url).content
# # tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
# tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
# print(tap_output)

In [ ]:
# print(tap_output['Gene'].unique())
# print(len(tap_output['Gene'].unique()))
# print(baseline_output['Gene'].unique())
# print(len(baseline_output['Gene'].unique()))

In [ ]:
# for x in tap_output['Gene'].unique():
#     tap_output_gene=tap_output[tap_output['Gene']==x]
#     gene_tap_data=tap_output[tap_output['Date'].isin(tap_output_gene['Date'].unique())]
#     gene_tap_data_final=gene_tap_data[gene_tap_data['Gene'].isin(['N2', x])]
#     gene_tap_data_final['taps']=gene_tap_data_final['taps'].astype(int)


# for x in baseline_output['Gene'].unique():
#     baseline_output_gene=baseline_output[baseline_output['Gene']==x]
#     gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
#     gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
#     for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
#         TTest_Allele(x, a, baseline_output, b)


# Run Statistics (T-Test and sample-mean distance) on Data

In [ ]:
# print(baseline_output)

## Sample Mean Distance & Confidence Interval Calculation of Baseline Data
1. Calculate Mean

2. Calculate SEM

3. Calculate CI

4. Calculate Mean Distance From Control (Mean of Strain - Mean of Control)

In [ ]:
print(tap_data.columns)

In [8]:
baseline_output_byplate=baseline_output.groupby(by=['Plate_id',
                                                    'Date',
                                                    'Screen',
                                                    'dataset',
                                                    'Gene',
                                                    'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                                    'n',
                                                                                                    'Number',
                                                                                                   'Date',
                                                                                                    'Time',
                                                                                                    'Screen',
                                                                                                    'dataset',
                                                                                                    'Allele'])
print(baseline_output_byplate)
print('-'*30)
print(baseline_output_byplate.columns[1:])
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*30)
tap_data_byplate=tap_data.groupby(by=['Plate_id',
                                    'Date',
                                    'Screen',
                                    'dataset',
                                    'Gene',
                                    'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                    'Screen',
                                                                                    'dataset',
                                                                                    'Allele',
                                                                                   'Date',
                                                                                    'dist',
                                                                                    'plate',
                                                                                    'recov_dura',
                                                                                    'recov_prob',
                                                                                    'recov_speed'])


print(tap_data_byplate)
print('-'*30)
print(tap_data_byplate.columns)
print('-'*50)

print("allele output")
print('-'*50)
baseline_output_allele_byplate=baseline_output.groupby(by=['Plate_id',
                                                            'Date',
                                                            'Screen',
                                                            'dataset',
                                                            'Gene',
                                                            'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                                            'n',
                                                                                                            'Number',
                                                                                                            'Time',
                                                                                                            'Screen',
                                                                                                           'Date',
                                                                                                            'Gene',
                                                                                                            'Allele'])
print(baseline_output_allele_byplate)
print('-'*50)
print(baseline_output_allele_byplate.columns)
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*50)
tap_data_allele_byplate=tap_data.groupby(by=['Plate_id',
                                            'Date',
                                            'Screen',
                                            'dataset',
                                            'Gene',
                                            'Allele'],as_index=False).mean().drop(columns=['Plate_id',
                                                                                            'Screen',
                                                                                            'Gene',
                                                                                           'Date',
                                                                                            'Allele',
                                                                                            'dist',
                                                                                            'plate',
                                                                                            'recov_dura',
                                                                                            'recov_prob',
                                                                                            'recov_speed'])


print(tap_data_allele_byplate)
print('-'*50)
print(tap_data_allele_byplate.columns)

       Gene  Instantaneous Speed  Interval Speed      Bias     Width  \
0        N2             0.071012        0.075835  0.150125  0.084687   
1     ced-1             0.066342        0.070083  0.149105  0.088107   
2     ced-1             0.042747        0.048264  0.055563  0.085415   
3     ced-1             0.107864        0.086273  0.416429  0.080042   
4     ced-1             0.115607        0.088923  0.514944  0.087389   
..      ...                  ...             ...       ...       ...   
268  lgc-34             0.069017        0.080340  0.205929  0.114775   
269   vab-3             0.059317        0.066440  0.152330  0.104552   
270      N2             0.077907        0.080118  0.312448  0.108200   
271  lgc-34             0.068264        0.083968  0.212251  0.105617   
272   vab-3             0.083227        0.083403  0.262274  0.095135   

       Length      Area  Angular Speed  Aspect Ratio       Kink      Curve  \
0    1.138129  0.122129       3.948523      0.259845  45.

In [ ]:
# print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1'])

In [9]:
''' 
input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
return: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
        for how many phenotypes there are in the input
'''
def extract_phenotypes(df):
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df


''' 
input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
return: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
        for how many phenotypes there are in the input
'''
def extract_allele_phenotypes(df):
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df




'''
input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
        output from extract_phenotypes function

output: the same list of dataframes with additional calculated columns for:
                1. mean
                2. count
                3. std
                4. ci95_hi
                5. ci95_low
                6. ci95
'''
        # baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        # gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        # gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_MSD(list_of_dfs):
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=['Gene'], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95(calculate_1)
        N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
      
        
        # calculate_2.columns=['_'.join(col) for col in calculate_2.columns.values]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs
    


'''
input: df of 4 columns: index, mean, count, std

output: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

'''
def ci95(df):
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df



'''
input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
        output from extract_phenotypes function

output: the same list of dataframes with additional calculated columns for:
                1. mean
                2. count
                3. std
                4. ci95_hi
                5. ci95_low
                6. ci95
'''
        # baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        # gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        # gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_allele_MSD(list_of_dfs):
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=['dataset'], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95_allele(calculate_1)
        N2_data = calculate_2[calculate_2.index=='N2_XJ1']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
        # calculate_2.columns=['_'.join(col) for col in calculate_2.columns.values]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs
    
        
'''
input: df of 4 columns: index, mean, count, std

output: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

'''
def ci95_allele(df):
    for metric in df.columns.levels[0]:
        if metric == 'dataset':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df



In [10]:
# test1 = extract_phenotypes(baseline_output_byplate)
# print(test1)
list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_output_byplate))
print(list_baseline_MSD)
# test2 = extract_phenotypes(tap_data_byplate)
# print(test2)
list_tap_MSD=calculate_MSD(extract_phenotypes(tap_data_byplate))
print(list_tap_MSD)

[                    Instantaneous Speed                                    
                                   mean count       sem   ci95_hi   ci95_lo
Gene                                                                       
N2                             0.000000    57  0.004099  0.008212 -0.008212
acd-1xdelm-2                   0.034337     5  0.012477  0.068977 -0.000303
acd-1xdelm-2xdelm-1            0.036644     5  0.010984  0.067141  0.006148
alr-1                         -0.013400    18  0.001972 -0.009239 -0.017561
ced-1                          0.002253    30  0.004833  0.012138 -0.007631
ced-12                        -0.018417    12  0.004997 -0.007420 -0.029415
cnd-1                         -0.039882    18  0.002180 -0.035283 -0.044482
dyf-11                        -0.033216    30  0.002029 -0.029066 -0.037367
gcy-18                         0.005938     4  0.008678  0.033556 -0.021680
gcy-23                        -0.028251     9  0.002755 -0.021897 -0.034604
gcy-8      

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [ ]:
for a in list_baseline_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_baseline_MSD[0].columns.levels[0]:
        baseline_MSD=a
    else:
        baseline_MSD=baseline_MSD.join(a)

for a in list_tap_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_tap_MSD[0].columns.levels[0]:
        tap_MSD=a
    else:
        tap_MSD=tap_MSD.join(a)

In [ ]:
print(baseline_MSD)

In [ ]:
print(tap_MSD)

In [ ]:
print(baseline_MSD.columns.levels[0])
print("-"*30)
print(tap_MSD.columns.levels[0])

In [ ]:
test_df=list_baseline_MSD[-1]
# test_df['Crab']
test_df.columns.levels[0]

In [ ]:
combined_MSD = baseline_MSD.join(tap_MSD, on='Gene')
print(combined_MSD.columns.levels[0])


In [ ]:
combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})
print(combined_MSD.columns.levels[0])
print(len(combined_MSD.columns.levels[0]))

In [ ]:
combined_MSD=combined_MSD.reset_index()
combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
combined_MSD=combined_MSD.rename(columns={"Gene-": "Gene"})
combined_MSD['Screen']=Screen
print(combined_MSD)

## Allele-level SMD

In [ ]:
# print(baseline_output_allele_byplate[baseline_output_allele_byplate.dataset=='N2_XJ1'])
# print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_N2'])

In [ ]:
# test1=extract_allele_phenotypes(baseline_output_allele_byplate)
# print(test1)
list_allele_baseline_MSD=calculate_allele_MSD(extract_allele_phenotypes(baseline_output_allele_byplate))
print(list_allele_baseline_MSD)

# test2=extract_allele_phenotypes(tap_data_allele_byplate)
# print(test2[0][test2[0].dataset=='N2_N2'])
list_allele_tap_MSD=calculate_allele_MSD(extract_allele_phenotypes(tap_data_allele_byplate))
print(list_allele_tap_MSD)

In [ ]:
for a in list_allele_baseline_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_allele_baseline_MSD[0].columns.levels[0]:
        allele_baseline_MSD=a
    else:
        allele_baseline_MSD=allele_baseline_MSD.join(a)

for a in list_allele_tap_MSD:
    # b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if a.columns.levels[0] == list_allele_tap_MSD[0].columns.levels[0]:
        allele_tap_MSD=a
    else:
        allele_tap_MSD=allele_tap_MSD.join(a)

In [ ]:
print(allele_baseline_MSD.columns.levels[0])
print("-"*30)
print(allele_tap_MSD.columns.levels[0])

In [ ]:
allele_combined_MSD = allele_baseline_MSD.join(allele_tap_MSD, on='dataset')
print(allele_combined_MSD.columns.levels[0])


In [ ]:
allele_combined_MSD=allele_combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                                         "habit_prob": "Habituation of Respones Probability",
                                                         "habit_speed":"Habituation of Response Speed",
                                                         "init_dura": "Initial Response Duration",
                                                         "init_prob": "Initial Response Probability",
                                                         "init_speed": "Initial Response Speed",
                                                         "final_dura": "Final Response Duration",
                                                         "final_prob": "Final Response Probability",
                                                         "final_speed": "Final Response Speed",
                                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})
print(allele_combined_MSD.columns.levels[0])
print(len(allele_combined_MSD.columns.levels[0]))

In [ ]:
allele_combined_MSD=allele_combined_MSD.reset_index()
allele_combined_MSD.columns = allele_combined_MSD.columns.to_flat_index().str.join('-')
allele_combined_MSD=allele_combined_MSD.rename(columns={"dataset-": "dataset"})
allele_combined_MSD['Screen']=Screen
print(allele_combined_MSD)

### Allele-level T-stat analysis of baseline data

In [ ]:
list=baseline_output["dataset"].unique()
print(list)
print(len(list))
print(baseline_output)

In [ ]:
def TTest_Allele(Type, DF_ref, output):
    for a in DF_ref['dataset'].unique():
        # Tstat=ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele == "XJ1" or DF_ref.Allele == "N2"][Type],equal_var=False)[0]
        Tstat=ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
PD_baseline_instantspeed_T_allele=pd.DataFrame(columns = ["dataset","Instantaneous Speed"])
PD_baseline_intspeed_T_allele=pd.DataFrame(columns = ["dataset","Interval Speed"])
PD_baseline_bias_T_allele=pd.DataFrame(columns = ["dataset","Bias"])
PD_baseline_width_T_allele=pd.DataFrame(columns = ["dataset","Width"])
PD_baseline_length_T_allele=pd.DataFrame(columns = ["dataset","Length"])
PD_baseline_area_T_allele=pd.DataFrame(columns = ["dataset","Area"])
PD_baseline_angularspeed_T_allele=pd.DataFrame(columns = ["dataset","Angular Speed"])
PD_baseline_aspectratio_T_allele=pd.DataFrame(columns = ["dataset","Aspect Ratio"])
PD_baseline_kink_T_allele=pd.DataFrame(columns = ["dataset","Kink"])
PD_baseline_curve_T_allele=pd.DataFrame(columns = ["dataset","Curve"])
PD_baseline_crab_T_allele=pd.DataFrame(columns = ["dataset","Crab"])

list_baseline_Tstats_allele=[PD_baseline_instantspeed_T_allele,
                      PD_baseline_intspeed_T_allele,
                      PD_baseline_bias_T_allele,
                      PD_baseline_width_T_allele,
                      PD_baseline_length_T_allele,
                      PD_baseline_area_T_allele,
                      PD_baseline_angularspeed_T_allele,
                      PD_baseline_aspectratio_T_allele,
                      PD_baseline_kink_T_allele,
                      PD_baseline_curve_T_allele,
                      PD_baseline_crab_T_allele]

list_baseline_metrics=["Instantaneous Speed",
                       "Interval Speed",
                       "Bias",
                       "Width",
                       "Length",
                       "Area",
                       "Angular Speed",
                       "Aspect Ratio",
                       "Kink",
                       "Curve",
                       "Crab"]


In [ ]:
import tqdm
for x in baseline_output['dataset'].unique():
    # if x == "N2_XJ1" or x == "N2_N2":
    if x == "N2":
        pass
    else:
        baseline_output_gene=baseline_output[baseline_output['dataset']==x]
        gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2',x])]
        # gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2_XJ1', 'N2_N2',x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_baseline_metrics, list_baseline_Tstats_allele):
            TTest_Allele(a, gene_baseline_data_final, b)

In [ ]:
PD_baseline_Tstats_allele=pd.DataFrame()
for a in list_baseline_Tstats_allele:
    b=a.groupby(['dataset'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_baseline_Tstats_allele[0].columns.values[1]:
        PD_baseline_Tstats_allele=b
    else:
        PD_baseline_Tstats_allele=PD_baseline_Tstats_allele.join(b.iloc[:,1])
        
print(PD_baseline_Tstats_allele)

In [ ]:
PD_baseline_Tstats_allele=PD_baseline_Tstats_allele.set_index("dataset")
print(PD_baseline_Tstats_allele)
PD_baseline_Tstats_allele_sort=PD_baseline_Tstats_allele.sort_index()
print(PD_baseline_Tstats_allele_sort)

### Gene-level T-stat analysis of baseline data

In [ ]:
def TTest_Gene(Type, DF_ref, output):
    for a in DF_ref['Gene'].unique():
        Tstat=ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
PD_baseline_instantspeed_T=pd.DataFrame(columns = ["Gene","Instantaneous Speed"])
PD_baseline_intspeed_T=pd.DataFrame(columns = ["Gene","Interval Speed"])
PD_baseline_bias_T=pd.DataFrame(columns = ["Gene","Bias"])
PD_baseline_width_T=pd.DataFrame(columns = ["Gene","Width"])
PD_baseline_length_T=pd.DataFrame(columns = ["Gene","Length"])
PD_baseline_area_T=pd.DataFrame(columns = ["Gene","Area"])
PD_baseline_angularspeed_T=pd.DataFrame(columns = ["Gene","Angular Speed"])
PD_baseline_aspectratio_T=pd.DataFrame(columns = ["Gene","Aspect Ratio"])
PD_baseline_kink_T=pd.DataFrame(columns = ["Gene","Kink"])
PD_baseline_curve_T=pd.DataFrame(columns = ["Gene","Curve"])
PD_baseline_crab_T=pd.DataFrame(columns = ["Gene","Crab"])

list_baseline_Tstats=[PD_baseline_instantspeed_T,
                      PD_baseline_intspeed_T,
                      PD_baseline_bias_T,
                      PD_baseline_width_T,
                      PD_baseline_length_T,
                      PD_baseline_area_T,
                      PD_baseline_angularspeed_T,
                      PD_baseline_aspectratio_T,
                      PD_baseline_kink_T,
                      PD_baseline_curve_T,
                      PD_baseline_crab_T]

list_baseline_metrics=["Instantaneous Speed",
                       "Interval Speed",
                       "Bias",
                       "Width",
                       "Length",
                       "Area",
                       "Angular Speed",
                       "Aspect Ratio",
                       "Kink",
                       "Curve",
                       "Crab"]

In [ ]:
for x in baseline_output['Gene'].unique():
    if x == "N2":
        pass
    else:
        baseline_output_gene=baseline_output[baseline_output['Gene']==x]
        gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
        gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
            TTest_Gene(a, gene_baseline_data_final, b)
            

In [ ]:
PD_baseline_Tstats=pd.DataFrame()
for a in list_baseline_Tstats:
    b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_baseline_Tstats[0].columns.values[1]:
        PD_baseline_Tstats=b
    else:
        PD_baseline_Tstats=PD_baseline_Tstats.join(b.iloc[:,1])
        
print(PD_baseline_Tstats)

In [ ]:
PD_baseline_Tstats=PD_baseline_Tstats.set_index("Gene")
print(PD_baseline_Tstats)
PD_baseline_Tstats_sort=PD_baseline_Tstats.sort_index()
print(PD_baseline_Tstats_sort)

### T-stat analysis for tap-response data

In [ ]:
# PD_tap=tap_output
# PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
# PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
# PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")
# print(PD_first_tap)
# print(PD_last_three_taps)
# print(PD_recov_taps)
# PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
# print(PD_final_taps)

In [ ]:
# PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
# PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
# print(PD_final_taps)
# print(PD_first_tap)

In [ ]:
# PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
# PD_habit_levels = PD_habit_levels.dropna()
# print(PD_habit_levels)

In [ ]:
# PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
# PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
# PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# # PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])

In [ ]:
# PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
# PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
# PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y','taps','time'])
# PD_habituation = PD_habituation.dropna()
# print(PD_habituation)

In [ ]:
# PD_habituation['recovery_dura']=PD_habituation.init_dura-PD_habituation.recov_dura
# PD_habituation['recovery_prob']=PD_habituation.init_prob-PD_habituation.recov_prob
# PD_habituation['recovery_speed']=PD_habituation.init_speed-PD_habituation.recov_speed
# PD_habituation=PD_habituation.dropna()
# print(PD_habituation)

In [ ]:
print(tap_data)

In [ ]:
print(tap_data.columns.values)

## Allele level T-stat analysis of tap response data

In [ ]:
recovery_dura_allele=pd.DataFrame(columns = ["dataset","Recovery Duration"])
recovery_prob_allele=pd.DataFrame(columns = ["dataset","Recovery Probability"])
recovery_speed_allele=pd.DataFrame(columns = ["dataset","Recovery Speed"])
init_dura_allele=pd.DataFrame(columns = ["dataset","Initial Duration"])
init_prob_allele=pd.DataFrame(columns = ["dataset","Initial Probability"])
init_speed_allele=pd.DataFrame(columns = ["dataset","Initial Speed"])
final_dura_allele=pd.DataFrame(columns = ["dataset","Final Duration"])
final_prob_allele=pd.DataFrame(columns = ["dataset","Final Probability"])
final_speed_allele=pd.DataFrame(columns = ["dataset","Final Speed"])
hab_dura_allele=pd.DataFrame(columns = ["dataset","Habituation of Duration"])
hab_prob_allele=pd.DataFrame(columns = ["dataset","Habituation of Probability"])
hab_speed_allele=pd.DataFrame(columns = ["dataset","Habituation of Speed"])

list_tap_Tstats_allele = [recovery_dura_allele,
                   recovery_prob_allele,
                   recovery_speed_allele,
                   init_dura_allele,
                   init_prob_allele,
                   init_speed_allele,
                   final_dura_allele,
                   final_prob_allele,
                   final_speed_allele,
                   hab_dura_allele,
                   hab_prob_allele,
                   hab_speed_allele]
list_tap_metrics = ["recovery_dura",
                    "recovery_prob",
                    "recovery_speed",
                    "init_dura",
                    "init_prob",
                    "init_speed",
                    "final_dura",
                    "final_prob",
                    "final_speed",
                    "habit_dura",
                    "habit_prob",
                    "habit_speed"]
print('done')

In [ ]:
for x in tap_data['dataset'].unique():
    if x == "N2":
    # if x == "N2_N2" or x == "N2_XJ1":
        pass
    else:
        PD_habituation_allele=tap_data[tap_data['dataset']==x]
        gene_PD_habituation_data=tap_data[tap_data['Date'].isin(PD_habituation_allele['Date'].unique())]
        gene_PD_habituation_data_final=gene_PD_habituation_data[gene_PD_habituation_data['dataset'].isin(['N2', x])]
        # gene_PD_habituation_data_final=gene_PD_habituation_data[gene_PD_habituation_data['dataset'].isin(['N2_N2', "N2_XJ1", x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_tap_metrics, list_tap_Tstats_allele):
            TTest_Allele(a, gene_PD_habituation_data_final, b)

In [ ]:
PD_habituation_Tstats_allele=pd.DataFrame()
for a in list_tap_Tstats_allele:
    b=a.groupby(['dataset'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_tap_Tstats_allele[0].columns.values[1]:
        PD_habituation_Tstats_allele=b
    else:
        PD_habituation_Tstats_allele=PD_habituation_Tstats_allele.join(b.iloc[:,1])
        
print(PD_habituation_Tstats_allele)

In [ ]:
PD_habituation_Tstats_allele=PD_habituation_Tstats_allele.set_index("dataset")
print(PD_habituation_Tstats_allele)
PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()
print(PD_habituation_Tstats_allele_sorted)
print(PD_baseline_Tstats_allele_sort)

## Gene-level T-stat analysis of Tap response data

In [ ]:
recovery_dura=pd.DataFrame(columns = ["Gene","Recovery Duration"])
recovery_prob=pd.DataFrame(columns = ["Gene","Recovery Probability"])
recovery_speed=pd.DataFrame(columns = ["Gene","Recovery Speed"])
init_dura=pd.DataFrame(columns = ["Gene","Initial Duration"])
init_prob=pd.DataFrame(columns = ["Gene","Initial Probability"])
init_speed=pd.DataFrame(columns = ["Gene","Initial Speed"])
final_dura=pd.DataFrame(columns = ["Gene","Final Duration"])
final_prob=pd.DataFrame(columns = ["Gene","Final Probability"])
final_speed=pd.DataFrame(columns = ["Gene","Final Speed"])
hab_dura=pd.DataFrame(columns = ["Gene","Habituation of Duration"])
hab_prob=pd.DataFrame(columns = ["Gene","Habituation of Probability"])
hab_speed=pd.DataFrame(columns = ["Gene","Habituation of Speed"])

list_tap_Tstats = [recovery_dura,
                   recovery_prob,
                   recovery_speed,
                   init_dura,
                   init_prob,
                   init_speed,
                   final_dura,
                   final_prob,
                   final_speed,
                   hab_dura,
                   hab_prob,
                   hab_speed]
list_tap_metrics = ["recovery_dura",
                    "recovery_prob",
                    "recovery_speed",
                    "init_dura",
                    "init_prob",
                    "init_speed",
                    "final_dura",
                    "final_prob",
                    "final_speed",
                    "habit_dura",
                    "habit_prob",
                    "habit_speed"]
print('done')

In [ ]:
for x in tap_data['Gene'].unique():
    if x == "N2":
        pass
    else:
        PD_habituation_gene=tap_data[tap_data['Gene']==x]
        gene_PD_habituation_data=tap_data[tap_data['Date'].isin(PD_habituation_gene['Date'].unique())]
        gene_PD_habituation_data_final=gene_PD_habituation_data[gene_PD_habituation_data['Gene'].isin(['N2', x])]
        # print(gene_baseline_data_final['Gene'].unique())
        for a,b in zip(list_tap_metrics, list_tap_Tstats):
            TTest_Gene(a, gene_PD_habituation_data_final, b)

In [ ]:
PD_habituation_Tstats=pd.DataFrame()
for a in list_tap_Tstats:
    b=a.groupby(['Gene'], as_index=False).mean()
    # print(a.columns.values[1])
    if b.columns.values[1] == list_tap_Tstats[0].columns.values[1]:
        PD_habituation_Tstats=b
    else:
        PD_habituation_Tstats=PD_habituation_Tstats.join(b.iloc[:,1])
        
print(PD_habituation_Tstats)

In [ ]:
PD_habituation_Tstats=PD_habituation_Tstats.set_index("Gene")
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()
print(PD_habituation_Tstats)
print(PD_habituation_Tstats_sorted)
print(PD_baseline_Tstats_sort)

# Merging t-stat data into one dataset

## Gene-level

In [ ]:
combined_Tstats = pd.merge(PD_baseline_Tstats, PD_habituation_Tstats, on =['Gene'], how ='left')
combined_Tstats=combined_Tstats.sort_index()
# PD_combined_Tstats=PD_combined_Tstats.drop(index="N2")
# combined_Tstats=combined_Tstats.dropna()
print(combined_Tstats)
combined_Tstats_normalize=(combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()
print(combined_Tstats_normalize)
print(combined_Tstats_normalize[combined_Tstats_normalize.index=="N2"])
print(combined_Tstats_normalize.index)
combined_Tstats_normalize_2=combined_Tstats_normalize-combined_Tstats_normalize[combined_Tstats_normalize.index=="N2"].squeeze()
print(combined_Tstats_normalize_2[combined_Tstats_normalize_2.index=="N2"])

first_col=combined_Tstats.pop("Area")
combined_Tstats.insert(0,"Area",first_col)

first_col=combined_Tstats.pop("Length")
combined_Tstats.insert(0,"Length",first_col)

first_col=combined_Tstats.pop("Width")
combined_Tstats.insert(0,"Width",first_col)

first_col=combined_Tstats.pop("Angular Speed")
combined_Tstats.insert(5,"Angular Speed",first_col)

print(combined_Tstats)

combined_Tstats=combined_Tstats.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                 "Habituation of Probability": "Habituation of Respones Probability",
                                                 "Habituation of Speed":"Habituation of Response Speed",
                                                 "Initial Duration": "Initial Response Duration",
                                                 "Initial Probability": "Initial Response Probability",
                                                 "Initial Speed": "Initial Response Speed",
                                                 "Final Duration": "Final Response Duration",
                                                 "Final Probability": "Final Response Probability",
                                                 "Final Speed": "Final Response Speed",
                                                 "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                 "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                 "Recovery Speed": "Spontaneous Recovery of Response Speed"})

combined_Tstats_normalize_2=combined_Tstats_normalize_2.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                                         "Habituation of Probability": "Habituation of Respones Probability",
                                                                         "Habituation of Speed":"Habituation of Response Speed",
                                                                         "Initial Duration": "Initial Response Duration",
                                                                         "Initial Probability": "Initial Response Probability",
                                                                         "Initial Speed": "Initial Response Speed",
                                                                         "Final Duration": "Final Response Duration",
                                                                         "Final Probability": "Final Response Probability",
                                                                         "Final Speed": "Final Response Speed",
                                                                         "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                                         "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                                         "Recovery Speed": "Spontaneous Recovery of Response Speed"})
print(combined_Tstats_normalize_2)

first_col=combined_Tstats_normalize_2.pop("Area")
combined_Tstats_normalize_2.insert(0,"Area",first_col)

first_col=combined_Tstats_normalize_2.pop("Length")
combined_Tstats_normalize_2.insert(0,"Length",first_col)

first_col=combined_Tstats_normalize_2.pop("Width")
combined_Tstats_normalize_2.insert(0,"Width",first_col)

first_col=combined_Tstats_normalize_2.pop("Angular Speed")
combined_Tstats_normalize_2.insert(5,"Angular Speed",first_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Duration")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Duration",last_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Probability")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Probability",last_col)

last_col=combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Speed")
combined_Tstats_normalize_2.insert(22,"Spontaneous Recovery of Response Speed",last_col)

print(combined_Tstats_normalize_2)

combined_Tstats_melted=combined_Tstats.reset_index()
combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=['Gene'],
                               var_name='Metric',
                               value_name='T_score')
combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])
print(combined_Tstats_melted_sorted)

combined_Tstats_normalized_melted=combined_Tstats_normalize_2.reset_index()
combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=['Gene'],
                                           var_name='Metric',
                                           value_name='T_score')
print(combined_Tstats_normalized_melted)


## Allele level

In [ ]:
combined_Tstats_allele = pd.merge(PD_baseline_Tstats_allele, PD_habituation_Tstats_allele, on =['dataset'], how ='left')
combined_Tstats_allele=combined_Tstats_allele.sort_index()
# PD_combined_Tstats=PD_combined_Tstats.drop(index="N2")
# combined_Tstats=combined_Tstats_allele.dropna()
print("combined_Tstats_allele")
print(combined_Tstats_allele)
print("-"*55)
combined_Tstats_normalize_allele=(combined_Tstats_allele-combined_Tstats_allele.mean())/combined_Tstats_allele.std()
print("combined_Tstats_normalize_allele")
print(combined_Tstats_normalize_allele)
print("-"*55)
# print(combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2"])
print("N2_XJ1 Data")
print(combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2_XJ1"])
print("-"*55)
print("combined_Tstats_normalize_allele row labels")
print(combined_Tstats_normalize_allele.index)
print("-"*55)
# combined_Tstats_normalize_allele_2=combined_Tstats_normalize_allele-combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2"].squeeze()
combined_Tstats_normalize_allele_2=combined_Tstats_normalize_allele-combined_Tstats_normalize_allele[combined_Tstats_normalize_allele.index=="N2_XJ1"].squeeze()
# print(combined_Tstats_normalize_allele_2[combined_Tstats_normalize_allele_2.index=="N2"])
print("combined Tstats normalized N2_XJ1 data")
print(combined_Tstats_normalize_allele_2[combined_Tstats_normalize_allele_2.index=="N2_XJ1"])
print("-"*55)
first_col=combined_Tstats_allele.pop("Area")
combined_Tstats_allele.insert(0,"Area",first_col)

first_col=combined_Tstats_allele.pop("Length")
combined_Tstats_allele.insert(0,"Length",first_col)

first_col=combined_Tstats_allele.pop("Width")
combined_Tstats_allele.insert(0,"Width",first_col)

first_col=combined_Tstats_allele.pop("Angular Speed")
combined_Tstats_allele.insert(5,"Angular Speed",first_col)
print("combined_Tstats_allele")
print(combined_Tstats_allele)
print("-"*55)
combined_Tstats_allele=combined_Tstats_allele.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                             "Habituation of Probability": "Habituation of Respones Probability",
                                                             "Habituation of Speed":"Habituation of Response Speed",
                                                             "Initial Duration": "Initial Response Duration",
                                                             "Initial Probability": "Initial Response Probability",
                                                             "Initial Speed": "Initial Response Speed",
                                                             "Final Duration": "Final Response Duration",
                                                             "Final Probability": "Final Response Probability",
                                                             "Final Speed": "Final Response Speed",
                                                             "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                             "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                             "Recovery Speed": "Spontaneous Recovery of Response Speed"})

combined_Tstats_normalize_allele_2=combined_Tstats_normalize_allele_2.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                                                             "Habituation of Probability": "Habituation of Respones Probability",
                                                                                             "Habituation of Speed":"Habituation of Response Speed",
                                                                                             "Initial Duration": "Initial Response Duration",
                                                                                             "Initial Probability": "Initial Response Probability",
                                                                                             "Initial Speed": "Initial Response Speed",
                                                                                             "Final Duration": "Final Response Duration",
                                                                                             "Final Probability": "Final Response Probability",
                                                                                             "Final Speed": "Final Response Speed",
                                                                                             "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                                                             "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                                                             "Recovery Speed": "Spontaneous Recovery of Response Speed"})

print("combined_Tstats_normalize_allele_2")
print(combined_Tstats_normalize_allele_2)
print("-"*55)
first_col=combined_Tstats_normalize_allele_2.pop("Area")
combined_Tstats_normalize_allele_2.insert(0,"Area",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Length")
combined_Tstats_normalize_allele_2.insert(0,"Length",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Width")
combined_Tstats_normalize_allele_2.insert(0,"Width",first_col)

first_col=combined_Tstats_normalize_allele_2.pop("Angular Speed")
combined_Tstats_normalize_allele_2.insert(5,"Angular Speed",first_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Duration")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Duration",last_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Probability")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Probability",last_col)

last_col=combined_Tstats_normalize_allele_2.pop("Spontaneous Recovery of Response Speed")
combined_Tstats_normalize_allele_2.insert(22,"Spontaneous Recovery of Response Speed",last_col)

print(combined_Tstats_normalize_allele_2)

combined_Tstats_normalized_melted_allele=combined_Tstats_normalize_allele_2.reset_index()
combined_Tstats_normalized_melted_allele=pd.melt(combined_Tstats_normalized_melted_allele, id_vars=['dataset'],
                                                   var_name='Metric',
                                                   value_name='T_score')
print(combined_Tstats_normalized_melted_allele)


combined_Tstats_melted_allele=combined_Tstats_allele.reset_index()
combined_Tstats_melted_allele=pd.melt(combined_Tstats_melted_allele, id_vars=['dataset'],
                                       var_name='Metric',
                                       value_name='T_score')
print(combined_Tstats_melted_allele)
combined_Tstats_melted_allele_sorted=combined_Tstats_melted_allele.sort_values(by=['T_score'])
print(combined_Tstats_melted_allele_sorted)

In [ ]:
combined_Tstats_normalize_2['Screen']=Screen
combined_Tstats_normalize_allele_2['Screen']=Screen
combined_Tstats_normalized_melted['Screen']=Screen
combined_Tstats_normalized_melted_allele['Screen']=Screen

In [ ]:
print(tap_output)

In [ ]:
print(combined_MSD)

## sqlite3 upload data

## A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

In [ ]:
def read(table):
    result = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    return result

conn = sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
old_tap_output = read('tap_response_data')
old_tap_tstat_gene = read('tstat_gene_data')
old_tap_tstat_allele = read('tstat_allele_data')
old_gene_profile_data = read('gene_profile_data')
old_allele_profile_data = read('allele_profile_data')
# old_gene_MSD = read('gene_MSD')
# old_allele_MSD = read('allele_MSD')

conn.close()

In [ ]:
# print(old_tap_tstat_allele)
# # print(old_tap_tstat_gene.drop_duplicates())

In [ ]:
# # conn=sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
# old_tap_output.drop_duplicates().to_sql('tap_response_data', conn, if_exists='replace', index=False)
# # baseline_output.to_sql('tap_baseline_data', conn, if_exists='replace', index=False)
# old_tap_tstat_gene.drop_duplicates().to_sql('tstat_gene_data', conn, if_exists='replace', index=False)
# old_tap_tstat_allele.drop_duplicates().to_sql('tstat_allele_data', conn, if_exists='replace', index=False)
# old_gene_profile_data.drop_duplicates().to_sql('gene_profile_data', conn, if_exists='replace', index=False)
# old_allele_profile_data.drop_duplicates().to_sql('allele_profile_data', conn, if_exists='replace', index=False)
# # combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)
# # allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)
# # combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
# print(conn.total_changes)
# conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
# cur.execute("DROP TABLE IF EXISTS tap_baseline_data")
# cur.execute("DROP TABLE IF EXISTS allele_phenotype_data")
# cur.execute("DROP TABLE IF EXISTS tstat_data")

# print("-"*60)

# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
# cur.execute(".tables")
conn.close()

In [ ]:
'''
input: (pd.DataFrame, pd.Dataframe or None, str)
    1 or 2 pd.DataFrames with the same indexes (columns), and sql table name (str)
    first DataFrame is old DataFrame read from the .db file
    second DataFrame is new processed DataFrame to be appended to the old one
    

function: If only one DataFrame, replace the .db tabl with this one DataFrame
 Otherwise (when given two dataframes):
     1. concatenate new DataFrame to old DataFrame
     2. to_sql, if_exists = 'replace'

'''

def update_table(df1, df2=None, table_name):
    if df2=None:
        df1.to_sql(table_name, conn, if_exists='replace', index=False)
    else:
        new_df=pd.concat([df1, df2], ignore_index=True)
        new_df.to_sql(table_name, conn, if_exists='replace', index=False)
        

In [ ]:
tap_output
combined_Tstats_normalize_2
combined_Tstats_normalize_allele_2
combined_Tstats_normalized_melted
combined_Tstats_normalized_melted_allele
combined_MSD
allele_combined_MSD

In [ ]:
conn=sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)
# baseline_output.to_sql('tap_baseline_data', conn, if_exists='replace', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)
combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)
allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)
# combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
print(conn.total_changes)
conn.close()

# Want to test edge cases of pd.to_sql functionality#############

In [ ]:
conn=sqlite3.connect('/Users/Joseph/Desktop/NRSC510B/mwt_data.db')
tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)
# baseline_output.to_sql('tap_baseline_data', conn, if_exists='replace', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)

In [ ]:
print(conn.total_changes)

In [ ]:
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM table_name WHERE primary_key_column = ?", (value,))
exists = cursor.fetchone()[0] > 0

if exists:
    print("Primary key already exists in the table.")
else:
    print("Primary key does not exist in the table.")

In [ ]:
conn.close()

## Clustering Playtest

The below cells are for exploring clustering functionalities. Currently under development.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca.transform(X)

In [ ]:
pca=PCA()
pca.fit(X)


In [ ]:
components=pca.transform(X)

In [ ]:
print(components)

In [ ]:
import plotly.express as px

features = ["Instantaneous Speed", "Interval Speed", "Bias", "Width","Length"]

pca = PCA()
components = pca.fit_transform(X[features])

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,width=800, height=800,
    labels=labels,
    dimensions=range(4))
    # color=df["species"]
# fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
print(X)

In [ ]:
# print(old_tap_tstat_allele)


meta=old_tap_tstat_allele.dropna()[['dataset','Screen']]
X=old_tap_tstat_allele.dropna().drop(columns=['dataset','Screen'])
# print(meta)
# print(X)

print("-"*55)
filtered = old_tap_tstat_allele[old_tap_tstat_allele['Screen']=="PD_Screen"]
filtered_meta = filtered.dropna()[['dataset','Screen']]
filtered_X = filtered.dropna().drop(columns=['dataset','Screen'])

# print(filtered_X)

In [ ]:
# print(old_tap_tstat_allele)
import numpy as np
filtered=old_tap_tstat_allele.drop(columns=['Spontaneous Recovery of Response Duration',
                                          'Spontaneous Recovery of Response Probability','Spontaneous Recovery of Response Speed'])
filtered_no_na=filtered.dropna()
print(filtered_no_na)
meta=filtered_no_na[['dataset','Screen']]
X = filtered_no_na.drop(columns=['dataset','Screen'])
print(np.unique(filtered_no_na.Screen))

In [ ]:
import seaborn as sns
figure=sns.pairplot(old_tap_tstat_gene.drop("Gene", axis=1), hue='Screen')
figure.savefig('/Users/Joseph/Desktop/MWT_Datasets/pairpot.png', bbox_inches="tight", dpi=450) 

In [ ]:
import umap
reducer = umap.UMAP()

In [ ]:
embedding = reducer.fit_transform(X)
embedding.shape

In [ ]:
fit = umap.UMAP(
        n_neighbors=10,
        min_dist=0.1,
        metric='euclidean'
        # n_components=n_components,
        # metric=metric
)
u = fit.fit_transform(X)
u.shape

In [ ]:
fit = umap.UMAP(
        n_neighbors=5,
        min_dist=0.1,
        metric='euclidean'
        # n_components=n_components,
        # metric=metric
)
filtered_u = fit.fit_transform(filtered_X)
filtered_u.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.scatter(
    u[:, 0],
    u[:, 1],
    c=[sns.color_palette()[x] for x in filtered_no_na.dropna().Screen.map({"PD_Screen":0, "ASD_Screen":1, "Glia_Genes_Screen":2, "G-Proteins_Screen":3, "Neuron_Genes_Screen":4})],
    )
plt.legend(loc="upper left")
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the MWT dataset', fontsize=24);

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(
    filtered_u[:, 0],
    filtered_u[:, 1],
    c=[sns.color_palette()[x] for x in filtered.dropna().Screen.map({"PD_Screen":0, "ASD_Screen":1, "Glia_Genes_Screen":2})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the MWT dataset', fontsize=24);